In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk, Value
import math
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
from transformers import Trainer, EarlyStoppingCallback

In [3]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification

In [4]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

In [5]:
sst_base = load_dataset('sst')

treebank_detok = TreebankWordDetokenizer()

sst = sst_base.map(
    lambda row: {
        "text": treebank_detok.detokenize(row["sentence"].split()),
        "label": min(math.floor(row["label"] / 0.2), 4.0),
    }).remove_columns(["tree", "tokens", "sentence" ])

sst_bin = sst_base.filter(
        lambda row: row["label"] < 0.4 or row["label"] >= 0.6
    ).map(
    lambda row: {
        "text": treebank_detok.detokenize(row["sentence"].split()),
        "label": min(math.floor(row["label"] / 0.5), 1.0),
    }).remove_columns(["tree", "tokens", "sentence" ])

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e1b5b68499f4feaa.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-aae790b7cb8bdaf4.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-360686b5b4f4ee4d.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f4f1ada73617d193.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6279b6f0f8a08f9a.arrow
Loading cached processed dataset at /home/jasko/.cache/huggi

In [6]:
yelp_bin_base = load_dataset('yelp_polarity')
split = yelp_bin_base['train'].train_test_split(0.001)
yelp_bin = DatasetDict({
    'train': split['train'],
    'validation': split['test'],
    'test': yelp_bin_base['test'],
})

Reusing dataset yelp_polarity (/home/jasko/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
yelp_full_base = load_dataset('yelp_review_full')
split = yelp_full_base['train'].train_test_split(0.001)
yelp_full = DatasetDict({
    'train': split['train'],
    'validation': split['test'],
    'test': yelp_full_base['test'],
})

Reusing dataset yelp_review_full (/home/jasko/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def tokenize_function(tokenizer, max_sequence_length, example):
    return tokenizer(example["text"],  truncation=True, padding="max_length", max_length=max_sequence_length )

In [9]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [10]:
def train(dataset, num_labels, base_model_checkpoint, problem_name, model_class, epochs=5):
    checkpoint = base_model_checkpoint
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    max_sequence_length = 128
    batch_size = 32
    eval_steps = 100
    learning_rate=2e-05
    num_train_epochs=epochs
    output_dir = "../output/"
    model_dir = "../models/"
    early_stopping_patience = 10

    tokenized_datasets = dataset.map(lambda example: tokenize_function(tokenizer, max_sequence_length, example), batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets = tokenized_datasets.cast_column("labels", Value("int32"))
    tokenized_datasets.set_format("torch")
    tokenized_datasets["train"].column_names

    model = model_class.from_pretrained(checkpoint, num_labels=5)

    training_args = TrainingArguments(
       output_dir+f"{base_model_checkpoint}-{problem_name}",
       evaluation_strategy ='steps',
       eval_steps = eval_steps , # Evaluation and Save happens every eval_steps steps
       save_total_limit = 1, # Only last  model is saved. Older ones are deleted.
       learning_rate=learning_rate,
       per_device_train_batch_size=batch_size,
       per_device_eval_batch_size=batch_size,
       num_train_epochs=num_train_epochs,
       metric_for_best_model = 'f1',
       load_best_model_at_end=True)


    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
        compute_metrics=compute_metrics,
    )


    trainer.train()

    model.save_pretrained(model_dir+f"{base_model_checkpoint}-{problem_name}")

    predictions = trainer.predict(tokenized_datasets["test"])
    print(predictions.predictions.shape, predictions.label_ids.shape)

    from sklearn.metrics import classification_report
    preds = np.argmax(predictions.predictions, axis=-1)

    print(classification_report(predictions.label_ids, preds))

In [11]:
train(sst_bin, 2, "bert-base-uncased", "sst_bin", BertForSequenceClassification)
train(sst_bin, 2, "roberta-base", "sst_bin", RobertaForSequenceClassification)
train(yelp_full, 5, "bert-base-uncased", "yelp_full", BertForSequenceClassification, epochs=1)
train(yelp_full, 5, "roberta-base", "yelp_full", RobertaForSequenceClassification, epochs=1)
train(yelp_bin, 2, "bert-base-uncased", "yelp_bin", BertForSequenceClassification, epochs=1)
train(yelp_bin, 2, "roberta-base", "yelp_bin", RobertaForSequenceClassification, epochs=1)

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-92569f49a6734dc4.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-3a1a23dc03ce4a16.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-a87e4ab4333ebb6d.arrow


Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-fc5cfc8935149adc.arrow
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (ini

...

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Runtime,Samples Per Second
100,No log,0.255401,0.907143,0.912504,0.910100,0.907100,2.795400,200.327000
200,No log,0.157267,0.939286,0.941149,0.937991,0.938974,2.799800,200.013000
300,No log,0.220534,0.932143,0.933337,0.933824,0.932139,2.799300,200.054000
400,No log,0.142916,0.939286,0.942135,0.937659,0.938909,2.801100,199.920000
500,0.258100,0.145979,0.946429,0.946666,0.945980,0.946274,2.798600,200.098000
600,0.258100,0.143401,0.942857,0.942568,0.943066,0.942769,2.800700,199.951000
700,0.258100,0.152410,0.939286,0.939051,0.939321,0.939174,2.801800,199.874000
800,0.258100,0.141233,0.948214,0.948352,0.947853,0.948075,2.801000,199.926000
900,0.258100,0.149171,0.946429,0.946127,0.946811,0.946360,2.800300,199.979000
1000,0.170100,0.136070,0.951786,0.951491,0.952097,0.951718,2.801200,199.913000


(38000, 5) (38000,)
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     19000
           1       0.96      0.95      0.95     19000

    accuracy                           0.95     38000
   macro avg       0.95      0.95      0.95     38000
weighted avg       0.95      0.95      0.95     38000

